# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

Note: Make sure that you have PostgreSQL running.

## Imports

In [828]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Generate Test Data

In [ ]:
# Create two blobs of data
prev_month_blobs = h.generate_random_polygons(n=10000)
curr_month_blobs = h.generate_random_polygons(n=10000)

Function `generate_random_polygons` executed in 8.5393 sec, CPU: 18.10%, Memory: 9.81MB


In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
1006,"POLYGON ((-98.04673 26.63848, -98.04673 26.715...",8a58013d-4251-4d66-8462-0d78d8268834,9udrcb6g
229,"POLYGON ((-97.92238 26.83128, -97.92238 26.926...",1b8a5a53-2cbd-4bb0-b2e8-1f99e23efc81,9uf2ujnt
404,"POLYGON ((-98.15002 27.11090, -98.15002 27.173...",460713ec-2d40-4468-8173-cd1ddb6e1769,9uf4rmp1
356,"POLYGON ((-98.19682 27.14905, -98.19682 27.246...",dfa95a81-4746-4b09-ad45-f4b00acee49b,9uf4wp4j
985,"POLYGON ((-97.96574 27.64931, -97.96574 27.711...",ecb64084-1bb7-4f7e-aae5-2df8c4beabb6,9ufm7x5d


In [ ]:
curr_month_blobs.head()

,geometry,id,geohash
4862,"POLYGON ((-80.81111 26.43806, -80.81111 26.455...",67815050-3edc-4b61-9985-ff3e4444d214,dhww3qkp
9493,"POLYGON ((-81.55671 26.58194, -81.55671 26.672...",72f96664-80de-4407-9764-ed10f12ef781,dhwp3p64
5335,"POLYGON ((-80.94495 26.84760, -80.94495 26.861...",20a0b1f5-51b5-4b61-892b-7d3f627e5a18,dhy2y29w
9086,"POLYGON ((-80.81058 26.85486, -80.81058 26.927...",6491576f-0882-4e90-9290-2faa1111a8a8,dhy8czru
7133,"POLYGON ((-81.39997 26.94769, -81.39997 26.970...",3010f2a4-b175-45f1-a905-1bab5880a96c,dhy1keu0


In [ ]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [ ]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
1006,"POLYGON ((-98.04673 26.63848, -98.04673 26.715...",8a58013d-4251-4d66-8462-0d78d8268834,9udrcb6g
229,"POLYGON ((-97.92238 26.83128, -97.92238 26.926...",1b8a5a53-2cbd-4bb0-b2e8-1f99e23efc81,9uf2ujnt
404,"POLYGON ((-98.15002 27.11090, -98.15002 27.173...",460713ec-2d40-4468-8173-cd1ddb6e1769,9uf4rmp1
356,"POLYGON ((-98.19682 27.14905, -98.19682 27.246...",dfa95a81-4746-4b09-ad45-f4b00acee49b,9uf4wp4j
985,"POLYGON ((-97.96574 27.64931, -97.96574 27.711...",ecb64084-1bb7-4f7e-aae5-2df8c4beabb6,9ufm7x5d


## Non-optimized process

### 1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [ ]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [ ]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

Function `convert_col_to_string` executed in 0.0531 sec, CPU: 24.80%, Memory: 2.84MB
Function `convert_col_to_string` executed in 0.0482 sec, CPU: 50.00%, Memory: 0.00MB
                                               geometry  \
1006  POLYGON ((-98.046731 26.638475, -98.046731 26....   
229   POLYGON ((-97.922379 26.831276, -97.922379 26....   
404   POLYGON ((-98.150023 27.110902, -98.150023 27....   
356   POLYGON ((-98.196816 27.149046, -98.196816 27....   
985   POLYGON ((-97.965741 27.649309, -97.965741 27....   

                                        id   geohash  
1006  8a58013d-4251-4d66-8462-0d78d8268834  9udrcb6g  
229   1b8a5a53-2cbd-4bb0-b2e8-1f99e23efc81  9uf2ujnt  
404   460713ec-2d40-4468-8173-cd1ddb6e1769  9uf4rmp1  
356   dfa95a81-4746-4b09-ad45-f4b00acee49b  9uf4wp4j  
985   ecb64084-1bb7-4f7e-aae5-2df8c4beabb6  9ufm7x5d  


/Users/sra/files/projects/matching_optimization/helpers.py:306: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))
/Users/sra/files/projects/matching_optimization/helpers.py:306: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))


### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [ ]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0036 sec, CPU: 0.00%, Memory: 0.00MB
Function `df_itertuple` executed in 0.0023 sec, CPU: 0.00%, Memory: 0.00MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [ ]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.0353 sec, CPU: 0.00%, Memory: 0.00MB


Create and insert into tables:

In [ ]:
prev_month_blobs.head(1)

,geometry,id,geohash
1006,"POLYGON ((-98.04673 26.63848, -98.04673 26.715...",8a58013d-4251-4d66-8462-0d78d8268834,9udrcb6g


In [ ]:
prev_month_blobs_wkt[0]

('POLYGON ((-98.046731 26.638475, -98.046731 26.715038, -97.963237 26.715038, -97.963237 26.638475, -98.046731 26.638475))',
 '8a58013d-4251-4d66-8462-0d78d8268834',
 '9udrcb6g')

In [ ]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.
Inserted 10000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 0.4146 sec, CPU: 0.00%, Memory: 0.00MB
Table curr_blobs_wkt truncated.
Inserted 10000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 0.3800 sec, CPU: 39.80%, Memory: 104.05MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [ ]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 10000 records from prev_blobs_wkt.
Function `retrieve_pg_table` executed in 0.1213 sec, CPU: 40.00%, Memory: 29.67MB
Retrieved 10000 records from curr_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0445 sec, CPU: 0.00%, Memory: 6.31MB


Compare the tables before and after for a sanity check:

In [ ]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-98.046731 26.638475, -98.046731 26....",8a58013d-4251-4d66-8462-0d78d8268834,9udrcb6g
1,"POLYGON ((-97.922379 26.831276, -97.922379 26....",1b8a5a53-2cbd-4bb0-b2e8-1f99e23efc81,9uf2ujnt
2,"POLYGON ((-98.150023 27.110902, -98.150023 27....",460713ec-2d40-4468-8173-cd1ddb6e1769,9uf4rmp1
3,"POLYGON ((-98.196816 27.149046, -98.196816 27....",dfa95a81-4746-4b09-ad45-f4b00acee49b,9uf4wp4j
4,"POLYGON ((-97.965741 27.649309, -97.965741 27....",ecb64084-1bb7-4f7e-aae5-2df8c4beabb6,9ufm7x5d


In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
1006,"POLYGON ((-98.04673 26.63848, -98.04673 26.715...",8a58013d-4251-4d66-8462-0d78d8268834,9udrcb6g
229,"POLYGON ((-97.92238 26.83128, -97.92238 26.926...",1b8a5a53-2cbd-4bb0-b2e8-1f99e23efc81,9uf2ujnt
404,"POLYGON ((-98.15002 27.11090, -98.15002 27.173...",460713ec-2d40-4468-8173-cd1ddb6e1769,9uf4rmp1
356,"POLYGON ((-98.19682 27.14905, -98.19682 27.246...",dfa95a81-4746-4b09-ad45-f4b00acee49b,9uf4wp4j
985,"POLYGON ((-97.96574 27.64931, -97.96574 27.711...",ecb64084-1bb7-4f7e-aae5-2df8c4beabb6,9ufm7x5d


In [ ]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 10000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


It worked!

### 2. Matching

We will match the polygons using GeoPandas.

In [ ]:
postgresql_details = h.pg_details()
h.run_parallel_matching(table_prev='prev_blobs_wkt', 
                        table_curr='curr_blobs_wkt', 
                        output_table='matched_results', 
                        postgresql_details=postgresql_details, 
                        db_name='blob_matching', 
                        num_workers=4, 
                        batch_size=100)

Table matched_results created successfully.
Retrieved 10000 records from prev_blobs_wkt.
Function `run_parallel_matching` executed in 46.4196 sec, CPU: 69.20%, Memory: 840.50MB


In [ ]:
# testing logging for matching_geometries
df_prev = h._retrieve_pg_table(postgresql_details, 'blob_matching', "prev_blobs_wkt", log_enabled=True)
df_curr = h._retrieve_pg_table(postgresql_details, 'blob_matching', "curr_blobs_wkt", log_enabled=True)

# Use a small subset for testing
df_prev_sample = df_prev.sample(10, random_state=42)
df_curr_sample = df_curr.sample(10, random_state=42)

h.match_geometries(df_prev_sample, df_curr_sample)  # Check if this logs matches

Retrieved 10000 records from prev_blobs_wkt.
Retrieved 10000 records from curr_blobs_wkt.
Function `match_geometries` executed in 0.0052 sec, CPU: 21.40%, Memory: 0.47MB


[]

In [769]:
prev_month_blobs.to_file('data/prev_month_blobs.geojson')
curr_month_blobs.to_file('data/curr_month_blobs.geojson')

geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.